In [14]:
from py2neo import Graph
from neo4j import GraphDatabase
import networkx as nx
import neo4jupyter

neo_host = "neo4j"
neo_port = 7687

graph = Graph(f"bolt://{neo_host}:{neo_port}")

In [10]:
NB_SELECTED = 4

graph.run("""MATCH ()-[r:Dissed]->(target)
                RETURN target, count(r) AS num
                ORDER BY num DESC
                LIMIT $nb_selected
            """, parameters={"nb_selected": NB_SELECTED}).to_data_frame()[['target']]

,target
0,"{'name': 'Jones Naser', 'wikidata_id': 59}"
1,"{'name': 'Graham Aubrey', 'wikidata_id': 102}"
2,"{'name': 'Mathers Bruce', 'wikidata_id': 98}"
3,"{'name': 'Hitler Adolf', 'wikidata_id': 22}"


In [11]:
NB_SELECTED = 4

graph.run("""MATCH (artist)-[r:Dissed]->()
                RETURN artist, count(r) AS num
                ORDER BY num DESC
                LIMIT $nb_selected
            """, parameters={"nb_selected": NB_SELECTED}).to_data_frame()[['artist']]

,artist
0,"{'name': 'Ice Cube', 'wikidata_id': 122}"
1,"{'name': 'Nick Cannon feat. Suge Knight, Hitma..."
2,"{'name': 'Nick Cannon feat. The Black Squad', ..."
3,"{'name': 'Bizarre', 'wikidata_id': 121}"


In [19]:
neo4jupyter.init_notebook_mode()
#sub_g = graph.run("""
#    MATCH (artist)-[r:Dissed]->(target)
#    RETURN *""").to_subgraph()
#neo4jupyter.draw_subgraph(sub_g, options={"artist": "wikidata_id", "target": "name"})
neo4jupyter.draw(graph, options={"Artist": "name", "Target": "name"})

<IPython.core.display.Javascript object>